In [2]:
import tensorflow as tf
from tensorflow.python.framework import ops
import numpy as np

In [3]:
from keras.datasets import mnist #Keras is used only to get MNIST data
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
Y_train = Y_train.reshape(60000, 1).T 
b = np.zeros((60000, 10))
b[np.arange(60000), Y_train] = 1
Y_train = b.T #To convert to one hot matrix
print(Y_train.shape)
X_train = X_train.reshape(60000, 28, 28, 1)/255 #Divided by 255 to normalize the input data
X_train = X_train.reshape(X_train.shape[0], -1).T #We use -1 because numpy will itself figure out what -1 is by looking at length of tha array and the remaining dimensions
print(X_train.shape)
Y_test = Y_test.reshape(10000, 1).T
b = np.zeros((10000, 10))
b[np.arange(10000), Y_test] = 1
Y_test = b.T
print(Y_test.shape)
X_test = X_test.reshape(10000, 28, 28, 1)/255
X_test = X_test.reshape(X_test.shape[0], -1).T
print(X_test.shape)
learning_rate  =0.1

Using TensorFlow backend.


(10, 60000)
(784, 60000)
(10, 10000)
(784, 10000)


In [6]:
m = 60000
batch_size = 128
batchno = m // batch_size
beta = 0.9
X = tf.placeholder(tf.float32, [784, None] , name = "X")
Y = tf.placeholder(tf.float32, [10, None] , name = "Y")

In [7]:
W1 = tf.get_variable("W1", [64,784], initializer = tf.contrib.layers.xavier_initializer())
b1 = tf.get_variable("b1", [64,1], initializer = tf.zeros_initializer())
W2 = tf.get_variable("W2", [10, 64], initializer = tf.contrib.layers.xavier_initializer())
b2 = tf.get_variable("b2", [10, 1], initializer = tf.zeros_initializer())

ValueError: Variable W1 already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\Sayan\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1801, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\Users\Sayan\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 3300, in create_op
    op_def=op_def)
  File "C:\Users\Sayan\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py", line 507, in new_func
    return func(*args, **kwargs)


In [8]:
Z1 = tf.add(tf.matmul(W1, X), b1)                              
A1 = tf.nn.relu(Z1)                                             
Z2 = tf.add(tf.matmul(W2, A1), b2) 
logits = tf.transpose(Z2)
labels = tf.transpose(Y)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [9]:
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

In [10]:
init = tf.global_variables_initializer() # Used for initializing all the tensors

In [11]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(15):
        for j in range(batchno):
            begin = j * batch_size
            end = min(begin + batch_size, 60000) #To prevent tracebacks
            Xm = X_train[:, begin:end]
            Ym = Y_train[:, begin:end]
            m_batch = end - begin
            _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: Xm, Y: Ym})
            epoch_cost += minibatch_cost / batchno

NameError: name 'epoch_cost' is not defined

In [12]:
correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess = tf.Session()
with sess.as_default():
    
    print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
    print ("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))

FailedPreconditionError: Attempting to use uninitialized value b2
	 [[node b2/read (defined at <ipython-input-5-4d5d38b9b33b>:4) ]]

Caused by op 'b2/read', defined at:
  File "C:\Users\Sayan\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Sayan\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Sayan\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Sayan\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Sayan\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\Users\Sayan\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\Sayan\Anaconda3\lib\asyncio\base_events.py", line 539, in run_forever
    self._run_once()
  File